In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split

In [2]:
TRAIN_CSV_PATH = '/kaggle/input/resnet-final/train.csv'
TRAIN_IMAGES_DIR = '/kaggle/input/resnet-final/train/train'

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_EPOCHS = 1     # Adjust as needed.
LEARNING_RATE = 1e-4

In [3]:
def load_and_preprocess_image(file_path, img_size):
    """
    Given a file path, reads the image, decodes, resizes and normalizes it.
    A debug print is included so you can verify the file path.
    """
    
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, img_size)
    image = image / 255.0
    return image

In [4]:
def create_dataset(df, images_dir, batch_size, img_size, shuffle=True):
    """
    Creates a tf.data.Dataset from a pandas DataFrame.
    Assumes the DataFrame has columns 'image_name' (without extension) and 'target'.
    """
    def process_row(image_name, label):
        # Build the full file path by appending ".jpg"
        file_name = tf.strings.join([image_name, '.jpg'])
        file_path = tf.strings.join([images_dir, file_name], separator='/')
        image = load_and_preprocess_image(file_path, img_size)
        return image, label

    # Create a dataset from the image names and targets.
    ds = tf.data.Dataset.from_tensor_slices((df['image_name'].values, df['target'].values))
    ds = ds.map(process_row, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=1000, seed=42)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [5]:
def bottleneck_block(x, filters, stride=1, conv_shortcut=False, name=None):
    """
    A bottleneck residual block.
    filters: list of three integers, e.g. [64, 64, 256]
    """
    shortcut = x
    if conv_shortcut:
        shortcut = layers.Conv2D(filters[2], kernel_size=1, strides=stride,
                                 kernel_initializer='he_normal', use_bias=False,
                                 name=name + '_0_conv')(x)
        shortcut = layers.BatchNormalization(name=name + '_0_bn')(shortcut)
    
    # First 1x1 convolution
    x = layers.Conv2D(filters[0], kernel_size=1, strides=stride,
                      kernel_initializer='he_normal', use_bias=False,
                      name=name + '_1_conv')(x)
    x = layers.BatchNormalization(name=name + '_1_bn')(x)
    x = layers.Activation('relu', name=name + '_1_relu')(x)
    
    # Second 3x3 convolution
    x = layers.Conv2D(filters[1], kernel_size=3, strides=1, padding='same',
                      kernel_initializer='he_normal', use_bias=False,
                      name=name + '_2_conv')(x)
    x = layers.BatchNormalization(name=name + '_2_bn')(x)
    x = layers.Activation('relu', name=name + '_2_relu')(x)
    
    # Third 1x1 convolution
    x = layers.Conv2D(filters[2], kernel_size=1, strides=1,
                      kernel_initializer='he_normal', use_bias=False,
                      name=name + '_3_conv')(x)
    x = layers.BatchNormalization(name=name + '_3_bn')(x)
    
    # Add shortcut connection
    x = layers.Add(name=name + '_add')([x, shortcut])
    x = layers.Activation('relu', name=name + '_out')(x)
    return x

def resnet_stack(x, filters, blocks, stride1=2, name=None):
    """
    A stack of bottleneck blocks.
    """
    x = bottleneck_block(x, filters, stride=stride1, conv_shortcut=True, name=name + '_block1')
    for i in range(2, blocks + 1):
        x = bottleneck_block(x, filters, stride=1, conv_shortcut=False, name=name + f'_block{i}')
    return x

def build_resnet152(input_shape=(224, 224, 3), num_classes=1):
    """
    Builds the original ResNet-152 model.
    For binary classification, the final Dense layer uses a sigmoid activation.
    """
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution and pooling layers.
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same',
                      kernel_initializer='he_normal', use_bias=False,
                      name='conv1_conv')(inputs)
    x = layers.BatchNormalization(name='conv1_bn')(x)
    x = layers.Activation('relu', name='conv1_relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same', name='pool1_pool')(x)
    
    # conv2_x: 3 blocks, filters [64, 64, 256]
    x = resnet_stack(x, filters=[64, 64, 256], blocks=3, stride1=1, name='conv2')
    
    # conv3_x: 8 blocks, filters [128, 128, 512]
    x = resnet_stack(x, filters=[128, 128, 512], blocks=8, stride1=2, name='conv3')
    
    # conv4_x: 36 blocks, filters [256, 256, 1024]
    x = resnet_stack(x, filters=[256, 256, 1024], blocks=36, stride1=2, name='conv4')
    
    # conv5_x: 3 blocks, filters [512, 512, 2048]
    x = resnet_stack(x, filters=[512, 512, 2048], blocks=3, stride1=2, name='conv5')
    
    # Global average pooling and dense prediction layer.
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    outputs = layers.Dense(num_classes, activation='sigmoid',
                           kernel_initializer='he_normal', name='predictions')(x)
    
    model = models.Model(inputs, outputs, name='ResNet152')
    return model


In [6]:
def create_test_dataset(images_dir, batch_size, img_size):
    """
    Creates a tf.data.Dataset for the test images.
    Assumes test images are JPEG files in the specified directory.
    """
    test_files = sorted([f for f in os.listdir(images_dir) if f.lower().endswith('.jpg')])
    
    def process_file(file):
        file_path = tf.strings.join([images_dir, file], separator='/')
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, img_size)
        image = image / 255.0
        return image, file  # return the file name along with the image
    
    ds = tf.data.Dataset.from_tensor_slices(test_files)
    ds = ds.map(lambda file: process_file(file), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [7]:
# 1. Load CSV and ensure target is integer.
df = pd.read_csv(TRAIN_CSV_PATH)
df['target'] = df['target'].astype(int)

print("Total samples before filtering:", len(df))
# 2. Filter out rows where the corresponding image file does not exist.
def file_exists(row):
    file_path = os.path.join(TRAIN_IMAGES_DIR, row['image_name'] + '.jpg')
    return os.path.exists(file_path)

df = df[df.apply(file_exists, axis=1)]
print("Total samples after filtering missing files:", len(df))

# 3. Split the filtered data into train, validation, and test (70/15/15 split).
train_df, temp_df = train_test_split(df, test_size=0.30, stratify=df['target'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['target'], random_state=42)

print("Training samples:", len(train_df))
print("Validation samples:", len(val_df))
print("Test samples:", len(test_df))

# 4. Create tf.data.Datasets.
train_ds = create_dataset(train_df, TRAIN_IMAGES_DIR, BATCH_SIZE, IMG_SIZE, shuffle=True)
val_ds   = create_dataset(val_df, TRAIN_IMAGES_DIR, BATCH_SIZE, IMG_SIZE, shuffle=False)
test_ds  = create_dataset(test_df, TRAIN_IMAGES_DIR, BATCH_SIZE, IMG_SIZE, shuffle=False)

# (Optional) Verify one batch of training data.
for images, labels in train_ds.take(1):
    print("Training batch - images shape:", images.shape, "labels shape:", labels.shape)


Total samples before filtering: 33126
Total samples after filtering missing files: 32701
Training samples: 22890
Validation samples: 4905
Test samples: 4906
Training batch - images shape: (16, 224, 224, 3) labels shape: (16,)


In [8]:
# Set up multi-GPU training using MirroredStrategy. This automatically splits each batch among the GPUs.
strategy = tf.distribute.MirroredStrategy()
print("Number of devices:", strategy.num_replicas_in_sync)

with strategy.scope():
    model = build_resnet152(input_shape=IMG_SIZE + (3,), num_classes=1)
    model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Print the model architecture.
model.summary()

Number of devices: 2


Model: "ResNet152"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,096 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,384 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,384 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_bn         │ (None, 56, 56, 256)    │          1,024 │ conv2_block1_3_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn    

 Total params: 58,297,281 (222.39 MB)

 Trainable params: 58,145,857 (221.81 MB)

 Non-trainable params: 151,424 (591.50 KB)

In [9]:
# ============================
#           TRAINING
# ============================
checkpoint_cb = callbacks.ModelCheckpoint('resnet152_skin_lesions.keras', save_best_only=True,
                                            monitor='val_loss', mode='min')
earlystop_cb  = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=[checkpoint_cb, earlystop_cb])

1431/1431 ━━━━━━━━━━━━━━━━━━━━ 808s 498ms/step - accuracy: 0.9646 - loss: 0.1262 - val_accuracy: 0.9804 - val_loss: 0.1422


In [ ]:
# # ============================
# #           INFERENCE (TEST DATA)
# # ============================
# # Note: predictions are generated in the same order as the sorted file names.
# predictions = model.predict(test_ds)

# print("\nTest Inference Results:")
# # Loop over the test dataset to print predictions along with file names.
# for images, files in test_ds:
#     preds = model.predict(images)
#     preds_binary = (preds > 0.5).astype(int).flatten()
#     for f, pred in zip(files.numpy(), preds_binary):
#         label = 'malignant' if pred == 1 else 'benign'
#         print("Image:", f, "Predicted:", label)

In [13]:
from sklearn.metrics import accuracy_score

all_true = []
all_pred = []

print("\nTest Inference Results:")
for images, labels in test_ds:
    preds = model.predict(images)
    preds_binary = (preds > 0.5).astype(int).flatten()
    for true_val, pred in zip(labels.numpy(), preds_binary):
        true_label = 'malignant' if true_val == 1 else 'benign'
        pred_label = 'malignant' if pred == 1 else 'benign'
    all_true.extend(labels.numpy().flatten())
    all_pred.extend(preds_binary)

accuracy = accuracy_score(all_true, all_pred)
print("\nTest Accuracy:", accuracy)



Test Inference Results:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_true = []
all_pred = []

print("\nBatch-wise Test Inference Results:")
for batch_idx, (images, labels) in enumerate(test_ds):
    preds = model.predict(images)
    preds_binary = (preds > 0.5).astype(int).flatten()
    batch_true = labels.numpy().flatten()
    
    # Compute batch metrics
    batch_acc = accuracy_score(batch_true, preds_binary)
    batch_prec = precision_score(batch_true, preds_binary, zero_division=0)
    batch_rec = recall_score(batch_true, preds_binary, zero_division=0)
    batch_f1 = f1_score(batch_true, preds_binary, zero_division=0)
    
    all_true.extend(batch_true)
    all_pred.extend(preds_binary)

# Compute overall metrics
overall_acc = accuracy_score(all_true, all_pred)
overall_prec = precision_score(all_true, all_pred, zero_division=0)
overall_rec = recall_score(all_true, all_pred, zero_division=0)
overall_f1 = f1_score(all_true, all_pred, zero_division=0)

print("\nOverall Test Metrics:")
print(f"Overall Accuracy:  {overall_acc:.4f}")
print(f"Overall Precision: {overall_prec:.4f}")
print(f"Overall Recall:    {overall_rec:.4f}")
print(f"Overall F1 Score:  {overall_f1:.4f}")



Batch-wise Test Inference Results:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 